<a href="https://colab.research.google.com/github/syt1298/Data-extraction-from-patents/blob/main/Cosine_Similarity_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Fetching from PTAB

In [ ]:
import os
import requests
import datetime
import time

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Project\ Data/DATA\ 2014-15/Txt\ Files/
#%ls

/content/drive/MyDrive/Project Data/DATA 2014-15/Txt Files


In [ ]:
%pwd

'/content/drive/MyDrive/Project Data/DATA 2014-15/Txt Files'

In [ ]:
DIR  = '/content/drive/MyDrive/Project Data/DATA 2014-15/Txt Files'
Txts = sorted([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])


In [ ]:
import numpy as np

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

import regex as re
import pandas as pd

In [ ]:
Txt_train = []
Txt_test = []
Corpus = []
Query = []
App_Corpus = []
App_Query = []

In [ ]:
for Txt in Txts:
    f_obj = open('/content/drive/MyDrive/Project Data/DATA 2014-15/Txt Files/'+Txt,"r")
    TEXT = f_obj.read().replace("\n"," ")
    try:
        app_no = str(re.search("[0-9]{2}\/[0-9]{3},[0-9]{3}",TEXT).captures()[0]).strip()
    except:
        app_no = ""
    if Txt[:10]<"2015-12-01" :
        if app_no != "":
            App_Corpus.append(app_no)
            Txt_train.append(Txt)
            Corpus.append(TEXT)
    else:
        App_Query.append(app_no)
        Txt_test.append(Txt)
        Query.append(TEXT)

In [ ]:
len(Corpus)

616

In [ ]:
len(Query)

500

In [ ]:
len(Txt_test)

500

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(Corpus)

In [ ]:
Y = vectorizer.transform(Query)

In [ ]:
y_hat = np.matmul(X.toarray(),Y.toarray().transpose())

In [ ]:
priors = np.argwhere(y_hat.transpose()>0.55)

In [ ]:
prior_ref = dict()
for prior in priors:
    if prior[0] not in prior_ref:
        prior_ref[prior[0]] = []
    prior_ref[prior[0]].append(prior[1])

In [ ]:
print(np.min(y_hat),np.quantile(y_hat,0.25),np.mean(y_hat),np.median(y_hat),np.quantile(y_hat,0.75),np.max(y_hat))

0.0 0.19839571755744512 0.24823346888463643 0.2462727382560068 0.2967704783269334 0.7813572160307996


In [ ]:
np.quantile(y_hat,0.75) + 1.5 *(np.quantile(y_hat,0.75)-np.quantile(y_hat,0.25))

0.44433261948116576

In [ ]:
files = pd.DataFrame(columns=["Application Number","Doc Name","PriorArt App No","PriorArt Doc Name"])

In [ ]:
for i in range(len(Query)):
    if i in prior_ref:
        for prior in prior_ref[i]:
            files = files.append({"Application Number":App_Query[i],"Doc Name" :Txt_test[i],"PriorArt App No":App_Corpus[prior],"PriorArt Doc Name":Txt_train[prior]},ignore_index=True)
    files = files.append({"Application Number":App_Query[i],"Doc Name" :Txt_test[i],"PriorArt App No":pd.NA,"PriorArt Doc Name":pd.NA},ignore_index=True)

In [ ]:
display(files.head(50))

,Application Number,Doc Name,PriorArt App No,PriorArt Doc Name
0,"12/361,413",2015-12-01_2013000538_DECISION.pdf.txt,<NA>,<NA>
1,,2015-12-01_CBM2015-00098 Mot.pdf.txt,<NA>,<NA>
2,,2015-12-01_CBM2015-00101 Mot.pdf.txt,<NA>,<NA>
3,,2015-12-01_CBM2015-00102 Mot.pdf.txt,<NA>,<NA>
4,,2015-12-01_IPR2013-00540 FD-RH.pdf.txt,<NA>,<NA>
5,,2015-12-01_IPR2013-00541 FD-RH.pdf.txt,<NA>,<NA>
6,,2015-12-01_IPR2013-00542 FD-RH.pdf.txt,<NA>,<NA>
7,,2015-12-01_IPR2013-00543 FD-RH.pdf.txt,<NA>,<NA>
8,,2015-12-01_IPR2013-00544 FD-RH.pdf.txt,<NA>,<NA>
9,,2015-12-01_IPR2013-00545 FD-RH.pdf.txt,<NA>,<NA>


In [ ]:
f_obj = open('/content/drive/MyDrive/Project Data/DATA 2012-13/Txt Files/2012-01-03_2010004696_DECISION.pdf.txt',"r")
TEXT = f_obj.read().replace("\n"," ")

'UNITED STATES PATENT AND TRADEMARK OFFICE  UNITED STATES DEPARTMENT OF COMMERCE United States Patent and Trademark Office Address: COMMISSIONER FOR PATENTS  P.O. Box 1450 Alexandria, Virginia 22313-1450 www.uspto.gov  APPLICATION NO.  FILING DATE  FIRST NAMED INVENTOR  ATTORNEY DOCKET NO.  CONFIRMATION NO.  11/414,101  04/28/2006  Susan M. Trefethren  64049744US01 (22312)  3559  7590  01/04/2012  23556 KIMBERLY-CLARK WORLDWIDE, INC. Tara Pohlkotte 2300 Winchester Rd. NEENAH, WI 54956  EXAMINER  OHERN, BRENT T  ART UNIT  PAPER NUMBER  1783  MAIL DATE  DELIVERY MODE  01/04/2012  PAPER  Please find below and/or attached an Office communication concerning this application or proceeding.  The time period for reply, if any, is set in the attached communication.  PTOL-90A (Rev. 04/07)  \x0cUNITED STATES PATENT AND TRADEMARK OFFICE  ____________________    BEFORE THE BOARD OF PATENT APPEALS  AND INTERFERENCES  ____________________      Ex parte SUSAN M. TREFETHREN, JONATHAN K.  ARENDT, CARL G

In [ ]:
print(TEXT.replace(chr(12)," \n "))

UNITED STATES PATENT AND TRADEMARK OFFICE  UNITED STATES DEPARTMENT OF COMMERCE United States Patent and Trademark Office Address: COMMISSIONER FOR PATENTS  P.O. Box 1450 Alexandria, Virginia 22313-1450 www.uspto.gov  APPLICATION NO.  FILING DATE  FIRST NAMED INVENTOR  ATTORNEY DOCKET NO.  CONFIRMATION NO.  11/414,101  04/28/2006  Susan M. Trefethren  64049744US01 (22312)  3559  7590  01/04/2012  23556 KIMBERLY-CLARK WORLDWIDE, INC. Tara Pohlkotte 2300 Winchester Rd. NEENAH, WI 54956  EXAMINER  OHERN, BRENT T  ART UNIT  PAPER NUMBER  1783  MAIL DATE  DELIVERY MODE  01/04/2012  PAPER  Please find below and/or attached an Office communication concerning this application or proceeding.  The time period for reply, if any, is set in the attached communication.  PTOL-90A (Rev. 04/07)   
 UNITED STATES PATENT AND TRADEMARK OFFICE  ____________________    BEFORE THE BOARD OF PATENT APPEALS  AND INTERFERENCES  ____________________      Ex parte SUSAN M. TREFETHREN, JONATHAN K.  ARENDT, CARL G. 